# 📈 Sequence Models - LSTM, GRU, CNN

## Objective

This notebook implements **sequence models** to predict burnout from 7-day temporal windows. Unlike MLP (which uses aggregated features), here the model sees the **sequence day by day**.

### Why Sequence Models?
Burnout develops over time. A single day doesn't capture the trend:
- Progressive decline in sleep quality
- Stress accumulation week after week
- Cyclical work-recovery patterns

### Implemented Architectures

#### 1. LSTM (Long Short-Term Memory)
- Recurrent networks with long-term "memory"
- Gates (forget, input, output) control information flow
- Best for: patterns spanning multiple days

#### 2. GRU (Gated Recurrent Unit)
- Simplified version of LSTM
- Fewer parameters, often similar performance
- Best for: smaller datasets or faster training

#### 3. CNN 1D (Convolutional Neural Network)
- Convolutional filters on temporal sequences
- Captures local patterns (2-3 consecutive days)
- Best for: repeated local patterns

### Input
- `data/processed/daily_with_burnout.parquet` (daily data with burnout label)

### Output
- `models/saved/lstm_classifier.pt`, `gru_classifier.pt`, `cnn1d_classifier.pt`

In [ ]:
# =============================================================================
# IMPORTS AND CONFIGURATION
# =============================================================================

from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

# Device selection
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Paths
DATA_DIR = Path('../data/processed')
MODEL_DIR = Path('../models/saved')
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# =============================================================================
# LOAD DAILY DATA
# =============================================================================
# Load daily data with burnout_level already calculated
# (generated by scripts/create_burnout_labels.py)

daily_path = DATA_DIR / 'daily_with_burnout.parquet'
daily = pd.read_parquet(daily_path)

# Ensure data is sorted by user and date
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.sort_values(['user_id', 'date'])

# IMPORTANT: Convert work_pressure from string to numeric
# Original dataset has values "low", "medium", "high"
if daily['work_pressure'].dtype == object:
    pressure_map = {'low': 0, 'medium': 1, 'high': 2}
    daily['work_pressure'] = daily['work_pressure'].map(pressure_map).fillna(1).astype(np.float32)

# Features for sequence model (15 daily metrics)
feature_cols = [
    'sleep_hours', 'sleep_quality',      # Sleep
    'work_hours', 'meetings_count',      # Work
    'tasks_completed', 'exercise_minutes', 'steps_count',  # Activity
    'caffeine_mg', 'alcohol_units', 'screen_time_hours',  # Consumption
    'stress_level', 'mood_score', 'energy_level', 'focus_score',  # Psychological
    'work_pressure'  # Environment (now numeric)
]

# Window size: 7 days (a work week)
window = 7
print(f"Features: {len(feature_cols)}, Window: {window} days")

In [ ]:
# =============================================================================
# SEQUENCE CREATION (SLIDING WINDOW)
# =============================================================================
# For each user, create sliding windows of 7 days.
# Label is the burnout of the LAST day of the window.
#
# Example for a user with 10 days:
#   Seq 1: days 1-7  → label = day 7
#   Seq 2: days 2-8  → label = day 8
#   Seq 3: days 3-9  → label = day 9
#   Seq 4: days 4-10 → label = day 10

def build_sequences(df, features, window):
    """
    Create sliding window sequences from daily data.
    
    Args:
        df: DataFrame with daily data
        features: list of feature columns
        window: window size in days
    
    Returns:
        X: array (N_seq, window, N_features)
        y: array (N_seq,) with labels
    """
    sequences, labels = [], []
    
    for uid, group in df.groupby('user_id'):
        feats = group[features].to_numpy(dtype=np.float32)
        labs = group['burnout_level'].to_numpy(dtype=np.int64)
        
        # Skip users with fewer than `window` days
        if len(group) < window:
            continue
        
        # Sliding window
        for idx in range(window, len(group) + 1):
            seq = feats[idx - window: idx]  # 7 days of features
            label = labs[idx - 1]            # Burnout of last day
            
            # Skip if there are NaN values
            if np.isnan(seq).any():
                continue
            
            sequences.append(seq)
            labels.append(label)
    
    return np.stack(sequences), np.array(labels)

# Build sequences
seq_X, seq_y = build_sequences(daily, feature_cols, window)
print(f"Total sequences: {len(seq_X):,}")
print(f"Shape: {seq_X.shape} = (sequences, days, features)")

In [ ]:
# =============================================================================
# TRAIN/VAL SPLIT AND DATALOADER
# =============================================================================

# Stratified split to maintain class proportions
X_train, X_val, y_train, y_val = train_test_split(
    seq_X, seq_y, test_size=0.2, stratify=seq_y, random_state=42
)

# Wrap in TensorDataset
train_ds = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_ds = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

# DataLoader with batching
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=256, shuffle=False)

# Dimensions for models
seq_len = X_train.shape[1]   # 7
input_dim = X_train.shape[2]  # 15
num_classes = len(np.unique(seq_y))  # 3
print(f"seq_len={seq_len}, input_dim={input_dim}, num_classes={num_classes}")

In [ ]:
# =============================================================================
# MODEL ARCHITECTURES
# =============================================================================

class SequenceNet(nn.Module):
    """
    Recurrent network (LSTM or GRU) for sequence classification.
    
    Architecture:
        Input (batch, 7, 15) 
            → LSTM/GRU 2 layers (hidden=128)
            → Take last hidden state
            → LayerNorm → ReLU → Dropout → Linear(3)
    """
    def __init__(self, input_dim, hidden_dim=128, cell='lstm'):
        super().__init__()
        # Select recurrent cell type
        rnn_cls = nn.LSTM if cell == 'lstm' else nn.GRU
        
        # 2-layer RNN with dropout between layers
        self.rnn = rnn_cls(
            input_dim, hidden_dim, 
            batch_first=True,  # Input: (batch, seq, features)
            num_layers=2, 
            dropout=0.2
        )
        
        # Classification head
        self.head = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        out, _ = self.rnn(x)       # out: (batch, seq_len, hidden)
        last = out[:, -1, :]       # Take last timestep
        return self.head(last)


class CNN1D(nn.Module):
    """
    1D CNN for sequence classification.
    
    Architecture:
        Input (batch, 7, 15) → transpose → (batch, 15, 7)
            → Conv1d(64, kernel=3) → ReLU → BatchNorm
            → Conv1d(128, kernel=3) → ReLU → GlobalAvgPool
            → Dropout → Linear(3)
    
    Convolutional filters capture local patterns (2-3 days).
    """
    def __init__(self, input_dim, seq_len):
        super().__init__()
        self.conv = nn.Sequential(
            # Conv1d: input_dim channels → 64 channels
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            # 64 → 128 channels
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            # Global average pooling: (batch, 128, seq) → (batch, 128, 1)
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        # x: (batch, seq_len, features)
        # Conv1d expects: (batch, channels, seq_len)
        x = x.transpose(1, 2)
        feats = self.conv(x)
        return self.fc(feats)

In [ ]:
# =============================================================================
# TRAINING FUNCTION
# =============================================================================
# Generic function to train any sequence model

def train_model(model, train_loader, val_loader, epochs=40, lr=1e-3, name='model'):
    """
    Standard training loop for sequence models.
    
    Args:
        model: PyTorch model
        train_loader, val_loader: DataLoaders
        epochs: number of epochs
        lr: learning rate
        name: name for saving checkpoint
    
    Returns:
        history: dict with train/val loss per epoch
    """
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    best_val = float('inf')
    history = {'train': [], 'val': []}
    
    for epoch in range(1, epochs + 1):
        # === Training ===
        model.train()
        tr_losses = []
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
            tr_losses.append(loss.item())
        
        # === Validation ===
        model.eval()
        val_losses = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                val_losses.append(criterion(model(xb), yb).item())
        
        train_loss = np.mean(tr_losses)
        val_loss = np.mean(val_losses)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        
        # Save best model
        if val_loss < best_val:
            best_val = val_loss
            torch.save({
                'state_dict': model.state_dict(), 
                'feature_cols': feature_cols
            }, MODEL_DIR / f'{name}.pt')
        
        if epoch % 5 == 0:
            print(f"[{name}] Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}")
    
    return history

In [ ]:
# =============================================================================
# TRAINING ALL 3 MODELS
# =============================================================================
# Train LSTM, GRU and CNN1D to compare architectures

print("Training LSTM...")
hist_lstm = train_model(
    SequenceNet(input_dim, cell='lstm'), 
    train_loader, val_loader, 
    name='lstm_classifier'
)

print("\nTraining GRU...")
hist_gru = train_model(
    SequenceNet(input_dim, cell='gru'), 
    train_loader, val_loader, 
    name='gru_classifier'
)

print("\nTraining CNN1D...")
hist_cnn = train_model(
    CNN1D(input_dim, seq_len), 
    train_loader, val_loader, 
    name='cnn1d_classifier'
)

In [ ]:
# =============================================================================
# LEARNING CURVES
# =============================================================================
# Visual comparison of training progress for all 3 models

def plot_history(history, title):
    """Plot train/val loss."""
    plt.figure(figsize=(7, 4))
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Validation')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()

plot_history(hist_lstm, 'LSTM Training Curves')
plot_history(hist_gru, 'GRU Training Curves')
plot_history(hist_cnn, 'CNN1D Training Curves')

In [ ]:
# =============================================================================
# FINAL EVALUATION
# =============================================================================
# Load best models and compare metrics

def evaluate_model(model_path):
    """Load a model and compute predictions on validation set."""
    payload = torch.load(model_path, map_location=DEVICE, weights_only=False)
    state = payload['state_dict']
    name = model_path.stem
    
    # Reconstruct correct architecture
    if 'lstm' in name:
        model = SequenceNet(input_dim, cell='lstm')
    elif 'gru' in name:
        model = SequenceNet(input_dim, cell='gru')
    else:
        model = CNN1D(input_dim, seq_len)
    
    model.load_state_dict(state)
    model = model.to(DEVICE)
    model.eval()
    
    with torch.no_grad():
        xb = torch.from_numpy(X_val).to(DEVICE)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
    
    return preds

# Evaluate all models
print("=== SEQUENCE MODELS COMPARISON ===\n")
for model_name in ['lstm_classifier.pt', 'gru_classifier.pt', 'cnn1d_classifier.pt']:
    preds = evaluate_model(MODEL_DIR / model_name)
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average='macro')
    print(f"{model_name:25s} Accuracy: {acc:.4f}, F1 Macro: {f1:.4f}")